# Creating a mask


In [ ]:
from pathlib import Path

import numpy as np

from astropy.nddata import CCDData

import ccdproc as ccdp
from photutils import detect_sources
from astrowidgets import ImageWidget

## Current thoughts

Ways to make a mask:

+ `ccdmask`, IRAF-inspired, looks for pixels that stand out in ratio of flat frames. **Issue:** current example 2 has very short range of exposure times (1 - 1.2) which is likely not ideal.
+ High dark current pixels. In this set of images the pixel `(3016, 3930)` is an excellent example. it shows up as hot in the master dark, it shows up as hot in the 1000 sec dark exposure, and it turns into a "dark" pixel in the reduced images because dark is over-subtracted.
+ Cosmic ray detection. This finds single pixels (should these be kept???), actual cosmic rays, and, it turns out, odd things like stars that snuck into the darks 🤦‍♂️. **Issue:** There are stars in the darks for this night. Am inclined to keep using this night anyway and discuss/point them out.
+ Some combination of the above. There is *not* huge overlap between them. **See near end of this notebook for support of this.** Not sure how many of the single-pixel "cosmic rays" are really cosmic rays.

When to create the mask:

+ At the end. Adding the mask to each file takes up *a lot* of space and isn't really needed if the procedures in this guide are followed because extreme pixels are rejected during reduction anyway. 
+ Once you have everything in hand constructing a mask is easier.
+ Can't really do any overall mask (as opposed to frame-by-frame) because the cosmic ray mask varies from image to image (mostly).

Do you need to create a mask?

+ Beats me. Lots of people don't, I gather. Worst case they introduce noise into your measurements, but some of that can be eliminated by sigma clipping the result and/or tossing out extreme points.
+ OTOH, if you do have one you can sensibly reject points from either annulus or aperture (or exclude measurements when bad pixel is in aperture or whatever).

Cool things:

+ Using `photutils` to generate segmentation maps from the mask is awesome. Makes it much easier to assess the cosmic ray detection and to view big bad spots from `ccdmask`.
+ Couple that with the image viewer and joy is to be had!

In [ ]:
ex2_path = Path('example2-reduced')

ifc = ccdp.ImageFileCollection(ex2_path)
ifc.summary.remove_columns(['history', 'comment'])

In [ ]:
flats = (ifc.summary['imagetyp'] == 'FLAT') & (ifc.summary['combined'] != True)
ifc.summary[flats]

In [ ]:
first = ifc.summary['file'][flats][0]
last = ifc.summary['file'][flats][-1]

In [ ]:
ccd1 = CCDData.read(ex2_path / first)
ccd2 = CCDData.read(ex2_path / last)

In [ ]:
mask1 = ccdp.ccdmask(ccd1)

In [ ]:
iw = ImageWidget()

In [ ]:
mccd1 = CCDData(data=mask1, unit='adu')

In [ ]:
iw.load_array(mask1)

In [ ]:
iw

In [ ]:
ratio = ccd2.divide(ccd1)

In [ ]:
%%time
maskr = ccdp.ccdmask(ratio)

In [ ]:
maskr.sum(), mask1.sum()

In [ ]:
iw.load_array(maskr)

In [ ]:
iw.load_nddata(ratio)

In [ ]:
iw.center_on((3824, 2450))

In [ ]:
x, y = np.where(maskr)

In [ ]:
len(x)

In [ ]:
segm = detect_sources(maskr, 0.5, 2)

In [ ]:
segm.slices

In [ ]:
cmp = segm.cmap()

In [ ]:
cmp.colors

In [ ]:
#iw.load_array(segm.data)
#iw.load_array(maskr)
iw.load_array(dmask)
#iw.load_array(dark1000.data)

In [ ]:
# This is a hot pixel that clearly shows up as a dark spot in other images
# i.e. it is not well-modeled by the dark frames.
iw.center_on((3016, 3930))
iw.zoom_level = 6

In [ ]:
iw

In [ ]:
dark1000 = CCDData.read('master_dark_exposure_1000.0.fit.bz2')

In [ ]:
dark1000 = dark1000.divide(1000).multiply(1.5)

In [ ]:
dmask = dark1000.data[:, :4096] > 10

In [ ]:
dmask.sum()

In [ ]:
in_both = dmask | maskr

In [ ]:
in_both.sum()

In [ ]:
iw.load_array(in_both)

In [ ]:
in_both.sum() / 4096**2 * 100

In [ ]:
maskd_from_cm = ccdp.ccdmask(dark1000)

In [ ]:
maskd_from_cm.sum()

In [ ]:
kelt16_with_cr = CCDData.read(ex2_path / 'kelt-16-with-cr-mask.fits')

In [ ]:
cr_mask = kelt16_with_cr.mask

In [ ]:
cr_mask.sum()

In [ ]:
only_in_cr = cr_mask & ~maskr & ~dmask

In [ ]:
only_in_cr.sum()